In [ ]:
#import standard libraries

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import cumtrapz
import h5py

import tidynamics

In [ ]:
#job_name = '50454376'                           # '50454376', '50454377', '50454378'
job_name = '50630219'                           # '50628659', '50630218', '50630219'
data_dir = '/Volumes/_work/genius/scratch/'

In [ ]:
def meansqrdis(t, D):
    return 6 * D * t

def vel_acf_fit(t, p0, p1):
    return p0 * (1-np.exp(-p1*t))

In [ ]:
diff_coeff = np.empty([4,40])

for i in range(40):
    j = i+1
    
    with open('./params_dpd.csv') as f:
        line = f.readlines()[j]
        line = line.split(',')
        
    rho, a_ij = line[0], line[1]
    
    print('Density, rep force:', rho, a_ij)
    
    vacf = np.load(data_dir + 'dpd_diff_' + job_name + '[' + str(j) +']/dpd_sample_vacf.npy')
    msd = np.load(data_dir + 'dpd_diff_' + job_name + '[' + str(j) +']/dpd_sample_msd.npy')
    
    idx1 = np.searchsorted(msd[:,0], 10)
    idx2 = np.searchsorted(msd[:,0], 400)
    print('Indixes:', idx1, idx2)
    print('Time of indexes:', msd[idx1,0], msd[idx2,0])
    
    #print(np.shape(vacf))
    vel_acf_int = cumtrapz(vacf[:,2], x = vacf[:,0], initial = 0)
    D_vacf, pcov = curve_fit(vel_acf_fit,
                             vacf[:,0], vel_acf_int, 
                             sigma = 1/(np.sqrt(vacf[:,1]))
                             )

    #print(np.shape(msd))
    D_msd, pcov = curve_fit(meansqrdis, 
                            msd[idx1:idx2,0], msd[idx1:idx2,2] + msd[idx1:idx2,3] + msd[idx1:idx2,4], 
                            sigma = 1/(np.sqrt(msd[idx1:idx2,1]))
                            )

    plt.figure(figsize=(15,10))
    ax1 = plt.subplot(131)
    ax1.loglog(vacf[:,0],vacf[:,1], label='n of samples')
    ax1.loglog(vacf[:,0],vacf[:,2], 'o', label='VACF Espresso')
    ax1.loglog(vacf[:,0],vel_acf_int, 'o', label='fit w/ VACF Espresso')
    ax1.loglog(vacf[:,0],vel_acf_fit(vacf[:,0],*D_vacf), label='Espresso VACF')
    ax1.legend()

    ax2 = plt.subplot(132)
    ax2.axvline(x=msd[idx1,0], color='tab:red')
    ax2.axvline(x=msd[idx2,0], color='tab:red')
    ax2.loglog(msd[:,0],msd[:,1], label='n of samples')
    ax2.loglog(msd[:,0],msd[:, 2] + msd[:, 3] + msd[:, 4], 'o', label=' MSD Espresso')
    ax2.loglog(msd[:,0],meansqrdis(msd[:,0],D_msd), label='fit w/ Espresso MSD')
    ax2.legend()
    
    print('D from VACF, MSD:', 1/3*D_vacf[0]/10000, D_msd[0])
    
    h5file = h5py.File(data_dir + 'dpd_diff_' + job_name + '[' + str(j) +']/trajectory.h5', 'r')

    cutoff = 600
    weights = np.arange(2*cutoff, 0, -2)
    
    r = h5file['particles/atoms/position/value'][-cutoff:]
    vel = h5file['particles/atoms/velocity/value'][-cutoff:]
    image = h5file['particles/atoms/image/value'][-cutoff:]
    le_offset = h5file['particles/atoms/lees_edwards_offset/value'][-cutoff:]
    edges = h5file['particles/atoms/box/edges/'][:]
    r_time = h5file['particles/atoms/position/time'][-cutoff:]
    time = r_time-r_time[0]
    
    pos = r + image*edges[None,None,:]
    
    msd_traj = np.empty([cutoff, 10000])
    
    for k in range(10000):
        msd_traj[:,k] = tidynamics.msd(pos[:,1,:])

    msd_mean = np.mean(msd_traj, axis=1)
    
    idx1 = np.searchsorted(time, 0)
    idx2 = np.searchsorted(time, 200)
    print('Indixes:', idx1, idx2)
    print('Time of indexes:', time[idx1], time[idx2])
        
    D_msd_traj, pcov = curve_fit(meansqrdis, 
                            time[idx1:idx2], msd_mean[idx1:idx2],
                            sigma = 1/(np.sqrt(weights[idx1:idx2]))
                            )
    
    ax3 = plt.subplot(133)
    ax3.axvline(x=time[idx1], color='tab:red')
    ax3.axvline(x=time[idx2], color='tab:red')
    ax3.loglog(time, weights, label='n of samples')
    ax3.loglog(time,msd_mean,'o', label='MSD tidynamics')
    ax3.loglog(time,meansqrdis(time,D_msd_traj), label='fit w/ tidynamics')
    ax3.legend()
    
    print('D from tidynamics:', D_msd_traj[0])
    
    diff_coeff[:,i] = rho, a_ij, D_msd_traj[0], D_msd[0]
    

In [ ]:
np.save('diffcoeff_' + job_name, diff_coeff)
    
print(diff_coeff)
plt.figure(figsize=(15,10))
    
selection = [0, 5, 10, 15, 20, 25, 30, 35]
selection = np.array(selection)

ax1 =plt.subplot(121)
for i in range(5):
    ax1 =plt.subplot(121)
    ax1.plot(diff_coeff[1,selection], diff_coeff[2,selection], 'o:')
    ax1.set_title("D from tidynamics")
    ax2 =plt.subplot(122)
    ax2.plot(diff_coeff[1,selection], diff_coeff[3,selection], 'o:')
    ax2.set_title("D from espresso")
    selection += 1